# Hyper-personalization Through Product Reviews: Exploring New Market Opportunites

A multi-modal approach to generating user recommendations.

## Import Libraries and Define File Paths

In [ ]:
import os
import glob
import json
import scipy
import random
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import threadpoolctl
from tqdm import tqdm
import warnings
from collections import defaultdict
from implicit.als import AlternatingLeastSquares as ALS
from lightfm import LightFM
from lightfm.data import Dataset as LightFMDataset
from scipy.sparse import coo_matrix
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from lightfm.cross_validation import random_train_test_split
from math import log2 as log
from scipy.spatial import distance
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, KFold
from surprise.model_selection import train_test_split as train_test_split_surprise
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans
from surprise.prediction_algorithms.matrix_factorization import SVD
from implicit.evaluation import (AUC_at_k,
                                 mean_average_precision_at_k,
                                 ndcg_at_k, precision_at_k)
from implicit.evaluation import train_test_split
from unidecode import unidecode
import uuid

warnings.simplefilter(action='ignore', category=FutureWarning)

dir = "../dataset/utility"

if not os.path.exists(dir):
    os.mkdir(dir)

## Define Utility Functions and Dataset Generation Functions

In [ ]:
def clean_str(x):
    forbidden_chars = [',', '-', '?', '(', ')',
                       '~', '*', '.', '!']
    x = unidecode(x)
    x = '_'.join(x.replace('& ', '').split(' '))
    x = x.lower().strip()
    for c in forbidden_chars:
        x = x.replace(c, '')
    return x

def items_and_reviews_to_dataframe(json_data):
    products = []
    reviews = []
    asins = []
    asin_product_mapping = []
    for product_data in json_data:
        product = {}
        if ('body' not in product_data or 'reviews' not in product_data['body']
            or 'productInformation' not in product_data['body']):
            continue

        reviews_data = product_data['body'].get('reviews', [])
        product_name = product_data['body'].get('name', 'Unknown Product')
        product_data = product_data['body']
        asin = product_data['canonicalUrl'].split('/')[-1].lower()

        if not reviews_data or len(reviews_data) == 0:
            continue
        
        ignore = ['dimensions', 'country_of_origin', 'batteries_included',
                  'weight', 'height', 'size', 'model', 'manufacturer',
                  'specifications', 'voltage', 'volts', '12v', 'climate_pledge',
                  'capacity', 'number_of_items', 'import', 'lxwxh', 'product'
                  'included']

        product['ASIN'] = asin

        if asin not in asins:
            asin_product_mapping.append({
                'ASIN': asin,
                'name': product_name
            })
            asins.append(asin)
        breadcrumbs = product_data.get('breadCrumbs', [])
        for bc in breadcrumbs:
            name = clean_str(bc['name'])
            flag = True
            for ig in ignore:
                if ig in name:
                    flag = False
            if flag and '_' in name:
                name_list = name.split("_")
                for n in name_list:
                    product[n] = 1.0
            elif flag:
                product[name] = 1.0
        
        products.append(product)

        review = {}
        for r in reviews_data:
            review['ASIN'] = asin
            review['ProductName'] = clean_str(product_name)
            reviewer_link = r['reviewerLink'].split('/')[-1].split('.')[-1]
            if len(reviewer_link) < 1:
                reviewer_link = str(uuid.uuid4())
            review['reviewerID'] = r['reviewerName'] + '_' + reviewer_link
            review['price'] = product_data.get('rawPrice', 0)
            reviewRating = re.findall(r'(\d+\.\d+)', r['reviewRating'])
            reviewLocation = r['reviewDate'].split('on')[0].split(' in ')[-1].replace('the ', '')
            reviewDate = re.findall(r'on (.+)$', r['reviewDate'])
            reviewVotes = re.findall(r'(\d+)', r['reviewVotes'])
            review['image'] = product_data.get('mainImage', None)
            
            if reviewRating:
                review['customerReview'] = float(reviewRating[0])
            else:
                review['customerReview'] = np.nan
            if reviewDate:
                review['reviewDate'] = reviewDate[0]
            else:
                review['reviewDate'] = 'Unknown'
            if reviewLocation:
                review['reviewLocation'] = reviewLocation
            else:
                review['reviewLocation'] = 'Unknown'
            if reviewVotes:
                review['reviewVotes'] = reviewVotes[0]
            else:
                review['reviewVotes'] = 0
            reviews.append(review)
            review = {}
    all_reviews_df = pd.DataFrame(reviews)
    all_items_df = pd.DataFrame(products)
    asins_df = pd.DataFrame(asin_product_mapping)
    return all_reviews_df, all_items_df, asins_df

def get_all_json_data():
    base_dir = '../dataset/extracts/amazon'
    all_json_data = []
    for root, dirs, files in os.walk(base_dir):
        for dir in dirs:
            items_path = os.path.join(root, dir, 'items')
            if os.path.exists(items_path):
                json_files = glob.glob(os.path.join(items_path, '*.json'))
                
                for json_file in tqdm(json_files, desc=f'Loading JSON Files in {dir}'):
                    try:
                        with open(json_file, "r") as f:
                            all_json_data.append(json.load(f))
                    except json.JSONDecodeError:
                        print(f"Error loading JSON from file {json_file}: file is empty or not a valid JSON.")
                    except Exception as e:
                        print(f"Unexpected error loading JSON from file {json_file}: {e}")
    return all_json_data

def load_dataset():
    items_df = pd.read_csv(f"{dir}/itemset_preprocessed.csv")
    items_df.set_index("ASIN", inplace=True)
 
    df_utility = pd.read_csv(f"{dir}/utility_topn.csv", index_col=0)
    df_utility.set_index("reviewerID", inplace=True)

    reviews_df = pd.read_csv(f"{dir}/reviews.csv")
    reviews_df.set_index('reviewerID', inplace=True)
    
    asins_df = pd.read_csv(f"{dir}/asin_product_mapping.csv")
    asins_df.set_index('ASIN', inplace=True)

    print(f"shape of df_utility: {df_utility.shape}")
    print(f"shape of items_df: {items_df.shape}")
    print(f"shape of asins_df: {asins_df.shape}")
    return items_df, df_utility, reviews_df, asins_df

## Generate the Dataset and Preprocess

### Fetch all json data

In [ ]:
all_json_data = get_all_json_data()
all_reviews_df, all_items_df, asins_df = items_and_reviews_to_dataframe(all_json_data)
all_reviews_df = all_reviews_df.replace(np.nan, '', regex=True)
all_reviews_df.drop_duplicates(keep="first", inplace=True)
all_reviews_df.to_csv(f"{dir}/reviews.csv")
all_items_df.drop_duplicates(keep='first', inplace=True)
all_items_df.fillna(0, inplace=True)
asins_df.to_csv(f"{dir}/asin_product_mapping.csv", index=False)

### Preprocess itemset

In [ ]:
all_items_df.reset_index(inplace=True)
vc = all_items_df['ASIN'].value_counts()
to_merge = vc.loc[lambda x: x > 1].index.tolist()

all_items_df.set_index('ASIN', inplace=True)
merge_dicts = []
for asin in to_merge:
    merged_row = all_items_df.loc[asin].sum()
    merged_row = merged_row.drop('level_0')
    d = merged_row.to_dict()
    d['ASIN'] = asin
    merge_dicts.append(d)
    all_items_df.drop(asin, inplace=True)
    
merged_df = pd.DataFrame(merge_dicts)
merged_df.set_index('ASIN', inplace=True)
all_items_df = pd.concat([all_items_df, merged_df])
all_items_df.to_csv(f"{dir}/itemset_preprocessed.csv")

### Preprocess userbase

In [ ]:
user_ratings_df = all_reviews_df.drop([
    "reviewVotes", "reviewLocation", "reviewDate",
    "ProductName"], axis=1)

groupby_df = user_ratings_df.groupby('reviewerID')
freq = groupby_df.size().reset_index(name='count')
user_ratings_df = pd.merge(user_ratings_df, freq, on='reviewerID', how='left')
user_ratings_df = user_ratings_df.sort_values(['count'], ascending=False)
user_ratings_df = user_ratings_df[user_ratings_df["count"] >= 5]
topn_reviewers = pd.unique(user_ratings_df["reviewerID"])
user_ratings_df.set_index("reviewerID", inplace=True)
user_ratings_df = user_ratings_df.loc[topn_reviewers]
generic_reviewerIDs = user_ratings_df.groupby('reviewerID').count().sort_values('ASIN', ascending=False)[:8].index.tolist()
topn_reviewers = [r for r in topn_reviewers if r not in generic_reviewerIDs]
user_ratings_df = user_ratings_df.loc[topn_reviewers]

rows = []
columns = all_items_df.index.tolist()
columns.append("reviewerID")

for index, data in user_ratings_df.groupby('reviewerID'):
    row = {'reviewerID': index}
    for ind, d in data.iterrows():
        row[d['ASIN']] = d['customerReview']
    rows.append(row)

df_utility = pd.DataFrame(rows, columns=columns)
df_utility.to_csv(f"{dir}/utility_topn.csv")

## Load Dataset

In [ ]:
items_df, df_utility, reviews_df, asins_df = load_dataset()

# Try all algorithms
We try all algorithms and evaluate the quality of recommendations.
* Neighborhood-based collaborative filtering
    * KNNBasic
    * KNNWithMeans
    * SVD
* Latent-factor based collaborative filtering
    * Alternating Least Squares (ALS)
* Content-based collaborative filtering
    * User profile numeric aggregation with Euclidean distance

## Neighborhood-based Collaborative Filtering

From Surprise

In [ ]:
melted_user_df = df_utility.copy().reset_index().melt(
    'reviewerID', var_name='ASIN', value_name='rating').dropna()
reader = Reader(rating_scale=(0,5))
dataset = Dataset.load_from_df(melted_user_df, reader)
algos = [KNNBasic(verbose=False), KNNWithMeans(verbose=False), SVD()]

In [ ]:
metrics_list = []
for algo in algos:
    metrics = cross_validate(algo, dataset)
    metrics['mean_test_rmse'] = np.mean(metrics['test_rmse'])
    metrics['mean_test_mae'] = np.mean(metrics['test_mae'])
    metrics_list.append(metrics)

k = range(0,5)
plt.figure(figsize=(10, 6))
plt.plot(k, metrics_list[0]['test_rmse'], label='Test RMSE for KNNBasic')
plt.plot(k, metrics_list[1]['test_rmse'], label='Test RMSE for KNNBWithMeans')
plt.plot(k, metrics_list[2]['test_rmse'], label='Test RMSE for SVD')
plt.plot(k, metrics_list[0]['test_mae'], label='Test MAE for KNNBasic')
plt.plot(k, metrics_list[1]['test_mae'], label='Test RMSE for KNNWithMeans')
plt.plot(k, metrics_list[2]['test_mae'], label='Test RMSE for SVD')
plt.plot(k, metrics_list[0]['fit_time'], label='Test Fit time for KNNBasic')
plt.plot(k, metrics_list[1]['fit_time'], label='Test Fit time for KNNWithMeans')
plt.plot(k, metrics_list[2]['fit_time'], label='Test Fit time for SVD')
plt.plot(k, metrics_list[0]['test_time'], label='Test time for KNNBasic')
plt.plot(k, metrics_list[1]['test_time'], label='Test time for KNNWithMeans')
plt.plot(k, metrics_list[2]['test_time'], label='Test time for SVD')
plt.xlabel("K fold")
plt.ylabel("Score")
plt.legend(loc='lower right')
plt.title("Performance of KNNBasic and KNNWithMeans")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(8,4), layout='constrained')
method = ["KNNBasic", "KNNWithMeans", "SVD"]
ind = 0
for m in metrics_list:
    for k in m:
        if 'mean' in k:
            axes[ind].bar(k, m[k])
            axes[ind].set_title(method[ind])
    ind += 1
plt.show()

### NDCG, Precision, and Recall

Taken from https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py and https://github.com/NicolasHug/Surprise/issues/339

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions, recalls
kf = KFold(n_splits=5)

def get_ndcg(surprise_predictions, k_highest_scores=None):
    """ 
    Calculates the ndcg (normalized discounted cumulative gain) from surprise predictions, using sklearn.metrics.ndcg_score and scipy.sparse
  
    Parameters: 
    surprise_predictions (List of surprise.prediction_algorithms.predictions.Prediction): list of predictions
    k_highest_scores (positive integer): Only consider the highest k scores in the ranking. If None, use all. 
  
    Returns: 
    float in [0., 1.]: The averaged NDCG scores over all recommendations
  
    """
    from sklearn.metrics import ndcg_score
    from scipy import sparse
    
    uids = []
    iids = []
    uid_mapping = {}
    iid_mapping = {}

    iter = 0
    for s in surprise_predictions:
        if s.uid in uid_mapping:
            uids.append(uid_mapping[s.uid])
        else:
            uid_mapping[s.uid] = iter
            uids.append(iter)

        if s.iid in iid_mapping:
            iids.append(iid_mapping[s.iid])
        else:
            iid_mapping[s.iid] = iter
            iids.append(iter)

        iter += 1

    r_uis = [p.r_ui for p in surprise_predictions ]
    ests = [p.est for p in surprise_predictions ]
    
    assert(len(uids) == len(iids) == len(r_uis) == len(ests) )    
    
    sparse_preds = sparse.coo_matrix( (ests, (uids , iids )) )
    sparse_vals = sparse.coo_matrix( (r_uis, (uids , iids )) )
    
    dense_preds = sparse_preds.toarray()
    dense_vals = sparse_vals.toarray()
    
    return ndcg_score(y_true= dense_vals , y_score= dense_preds, k=k_highest_scores)

algo_res = {}
method = ["KNNBasic", "KNNWithMeans", "SVD"]
iter = 0
for algo in algos:
    res_dict = {'precision': [], 'recall': [], 'ndcg': []}
    for trainset, testset in kf.split(dataset):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)
        ndcgs = get_ndcg(predictions, 5)
        res_dict['precision'].append(sum(prec for prec in precisions.values()) / len(precisions))
        res_dict['recall'].append(sum(rec for rec in recalls.values()) / len(recalls))
        res_dict['ndcg'].append(ndcgs)
    algo_res[method[iter]] = res_dict
    iter += 1

df_metrics_surprise = pd.DataFrame.from_dict({(i,j): algo_res[i][j] 
                           for i in algo_res.keys() 
                           for j in algo_res[i].keys()},
                       orient='index', columns=['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5'])
df_metrics_surprise['average'] = df_metrics_surprise.apply(pd.Series.mean, axis=1)
display(df_metrics_surprise)

### Get TopN Recommendations For All Users

Taken from https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
algo_names = ['KNNBasic', 'KNNWithMeans', 'SVD']
recommendations_surprise = {'KNNBasic': {}, 'KNNWithMeans': {}, 'SVD': {}}
iter = 0
for algo in algos:
    trainset, testset = train_test_split_surprise(dataset, test_size=1.0)
    predictions = algo.test(testset)
    top_n = get_top_n(predictions, n=10)
    for uid, user_ratings in top_n.items():
        recommendations_surprise[algo_names[iter]][uid] = [iid for (iid, _) in user_ratings]
    iter += 1

### Pick out users and explore their history and recommendations

In [ ]:
def get_random_users(lst, n):
    rand_ids = random.sample(lst, k=n)
    return rand_ids


def get_ilocs(df_utility, userids):
    userids_ilocs = []
    original_indices = df_utility.index
    for u in userids:
        userids_ilocs.append(original_indices.get_loc(u))
    return userids_ilocs


userids = get_random_users(
    list(recommendations_surprise['SVD'].keys()),
    len(list(recommendations_surprise['SVD'].keys()))
)
userids_ilocs = get_ilocs(df_utility, userids)

## Latent-factor based Collaborative Filtering

### ALS

In [ ]:
csr_utility = scipy.sparse.csr_matrix(df_utility.fillna(0).values)
#!export OPENBLAS_NUM_THREADS=1
#threadpoolctl.threadpool_limits(1, "blas")

auc_list = []
mean_ave_precision_list = []
ndcg_list = []
precision_list = []

factors = [50, 100, 150, 200, 250, 300,
           350]

for factor in factors:
    model = ALS(
        factors=factor,
        regularization=0.05,
        alpha=1.5,
        use_cg=True,
        iterations=20,
        random_state=143
    )
    model.fit(csr_utility, show_progress=False)

    # evaluation
    training_set, test_set = train_test_split(
        csr_utility,
        train_percentage=0.85,
        random_state=143
    )
    auc = AUC_at_k(model,training_set,test_set,
                K=len(df_utility), show_progress=True)
    mean_ave_precision = mean_average_precision_at_k(model, training_set,
                test_set, K=len(df_utility), show_progress=True)
    ndcg = ndcg_at_k(model, training_set, test_set,
                K=len(df_utility), show_progress=True)
    precision = precision_at_k(model, training_set,
                test_set, K=len(df_utility), show_progress=True)
    auc_list.append(auc)
    mean_ave_precision_list.append(mean_ave_precision)
    ndcg_list.append(ndcg)
    precision_list.append(precision)

plt.plot(factors, auc_list, label='AUC')
plt.plot(factors, mean_ave_precision_list, label='Mean Average Precision')
plt.plot(factors, ndcg_list, label='NDCG')
plt.plot(factors, precision_list, label='Precision')
plt.xlabel('Number of latent factors')
plt.ylabel('Performance score')
plt.title("Performance of ALS across number of factors")
plt.legend()
plt.show()

Select `factors=250`

In [ ]:
model = ALS(
    factors=250,
    regularization=0.05,
    alpha=1.5,
    calculate_training_loss=False,
    use_cg=True,
    iterations=20,
    random_state=143
)
model.fit(csr_utility, show_progress=False)

auc = AUC_at_k(model,training_set,test_set,
            K=len(df_utility), show_progress=True)
ndcg_als = ndcg_at_k(model, training_set, test_set,
            K=len(df_utility), show_progress=True)
precision = precision_at_k(model, training_set,
            test_set, K=len(df_utility), show_progress=True)

print(f"AUC: {auc}")
print(f"precision: {precision}")
print(f"ndcg: {ndcg_als}")

ids, scores = model.recommend(userids_ilocs, csr_utility[userids_ilocs])
recommendations_als_ilocs = [(x, y, z) for x, y, z in zip(userids_ilocs, ids, scores)]

recommendations_als = {}
for user_ind, reco_ids, scores in recommendations_als_ilocs:
    recos = {}
    reviewer_id = df_utility.iloc[user_ind, :].name
    reco_ids = df_utility.columns[reco_ids]
    n = 0
    for i in range(len(reco_ids)):
        recos[reco_ids[i]] = scores[i]
        n += 1
        if n == 9: break
    recommendations_als[reviewer_id]  = recos

## Content-based Recommender Systems

* Basic content-based recommender system using numerical user profile aggregation. Try cosine, euclidean, and jaccard distance metrics
* Hybrid recommender using LightFM

We first use a basic content-based recommender system that relies on aggregating the numeric user profile and various distance metrics such as cosine, euclidean, and jaccard. Then, we compare the results to a Hybrid Recommender System as provided by the LightFM library.

### Content-Based Recommender

* Use numerical user profile aggregation
* Try different distance metrics such as cosine, euclidean, and Jaccard distance

In [ ]:
def get_user_profile(user_ratings_nonzero, items_df):
    """
    Return series representing aggregated user profile
    """
    nonzero_asins = user_ratings_nonzero.index.tolist()
    user_profile = items_df.mul(user_ratings_nonzero, axis=0).sum(
        axis=0) / user_ratings_nonzero.notna().sum()
    return user_profile, nonzero_asins


def get_recommendations(user_profile, items_df,
        n, user_history, eval, dist):
    """
    Return dictionary of sorted recommendations
    """
    if dist == 'cos':
        dist_func = distance.cosine
    if dist == 'euc':
        dist_func = distance.euclidean
    if dist == 'jac':
        dist_func = distance.jaccard
    similarities = {}
    for ind, feats in items_df.iterrows():
        if ind not in user_history:
            similarities[ind] = np.abs(dist_func(feats, user_profile))
        if eval:
            similarities[ind] = np.abs(dist_func(feats, user_profile))
    if dist == 'euc':
        rev = False
    if dist == 'cos' or dist == 'jac':
        rev = True
    sorted_similarites = dict(sorted(
        similarities.items(), key=lambda item: item[1], reverse=rev)[:n])
    return sorted_similarites


def dcg(user_history, recommendations):
    """
    Return dcg for this user given their history
    and recommendations
    """
    ranking = 1
    dcg_final = 0
    actual_ratings = user_history.reset_index()[
        ['ASIN', 'customerReview']].set_index('ASIN').to_dict()['customerReview']
    for key, value in recommendations.items():
        dcg_final += ((2**actual_ratings[key]) - 1) / log(ranking + 1)
        ranking += 1
    return dcg_final / len(recommendations)


def idcg(user_history, recommendations):
    """
    Return ndcg for this user given their history
    and recommendations
    """
    actual_ratings = user_history.reset_index()[
        ['ASIN', 'customerReview']].set_index('ASIN').to_dict()['customerReview']
    ranking = 1
    idcg_final = 0
    for key in actual_ratings.keys():
        idcg_final += ((2**actual_ratings[key]) - 1) / log(ranking + 1)
        ranking += 1
    return idcg_final / len(recommendations)


def basic_content_based_recommender(
    userids, df_utility, items_df,
    reviews_df, n, eval, dist):
    """
    Return recommendations for the list of users.

    Parameters:
    userids: STRING list
    df_utility: DataFrame representing the utility matrix
    items_df: DataFrame representing the itemset
    reviews_df: DataFrame representing user history
    asins_df: DataFrame that contains mapping between ASINs and Product Names
    n: INTEGER Number of recommendations to output
    dist: STRING distance function to be used
    """
    recommendations = {}
    ndcg = None
    dcg_list = []
    idcg_list = []
    ndcg_list = []
    iter = 0
    for user in userids:
        user_ratings = df_utility.loc[user]
        user_ratings_nonzero = user_ratings[user_ratings > 0]
        user_profile, nonzero_asins = get_user_profile(user_ratings_nonzero, items_df)
        user_history_df = reviews_df.loc[user].drop([
            "Unnamed: 0", "reviewDate", "reviewLocation", "reviewVotes"
        ], axis=1)
        user_history_df = user_history_df.sort_values('customerReview', ascending=False)
        print(f"generating recos for {user} | {iter}")
        if eval:
            recos = get_recommendations(
                user_profile,
                items_df.loc[nonzero_asins],
                len(user_profile),
                reviews_df.loc[user]['ASIN'].tolist(),
                eval,
                dist
            )
            dcg_list.append(dcg(user_history_df, recos))
            idcg_list.append(idcg(user_history_df, recos))
        else:
            recos = get_recommendations(
                user_profile,
                items_df,
                n,
                reviews_df.loc[user]['ASIN'].tolist(),
                eval,
                dist
            )
        #print(f"sorted user history: {user}")
        #display(user_history_df)
        #print(f"recommended items")
        #df_reco = asins_df.loc[list(recos.keys())]
        #display(df_reco)
        recommendations[user] = recos
        iter += 1
    if eval:
        dcg_final = np.mean(dcg_list)
        idcg_final = np.mean(idcg_list)
        ndcg = dcg_final / idcg_final
        ndcg_list.append(ndcg)
    return recommendations, ndcg_list

## Evaluation: NDCG @ K

Get average NDCG across all users. Compare NDCG across different distance metrics.

In [ ]:
np.seterr(invalid='ignore')
ndcg_content = {}
items_df, df_utility, reviews_df, asins_df = load_dataset()
df_utility.fillna(0, inplace=True)
items_df.fillna(0, inplace=True)
recommendations_content = {}
try:
    items_df.drop(['level_0'], inplace=True, axis=1)
except:
    pass
try:
    reviews_df.set_index('reviewerID', inplace=True)
except:
    pass
try:
    asins_df.set_index('ASIN', inplace=True)
except:
    pass

# evaluate ndcg
recommendations, ndcg_list = basic_content_based_recommender(
    userids, df_utility, items_df.copy(),
    reviews_df, 10, True, 'euc')
ave_ndcg = np.mean(ndcg_list)
print(f"average ndcg with distance metric euc is {ave_ndcg}")

# generate actual recommendations
recommendations_content['euc'], _ = basic_content_based_recommender(
    userids, df_utility, items_df.copy(),
    reviews_df, 10, False, 'euc')

## Hybrid Recommender Using LightFM

A hybrid latent representation recommender model.

The model learns embeddings (latent representations in a high-dimensional space) for users and items in a way that encodes user preferences over items. When multiplied together, these representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

### LightFM Training Model

- Logistic: useful when both positive (1) and negative (-1) interactions are present.
- BPR: Bayesian Personalised Ranking 1 pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- WARP: Weighted Approximate-Rank Pairwise 2 loss. Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.
- k-OS WARP: k-th order statistic loss 3. A modification of WARP that uses the k-th positive example for any given user as a basis for pairwise updates.

In [ ]:
items_df, df_utility, reviews_df, asins_df = load_dataset()
melted_user_df = df_utility.copy().reset_index().melt(
    'reviewerID', var_name='ASIN', value_name='rating').dropna()
udf = melted_user_df

In [ ]:
dataset = LightFMDataset()
dataset.fit(users=udf['reviewerID'].unique(),
            items=udf['ASIN'].unique())

(interactions, _) = dataset.build_interactions(zip(udf['reviewerID'], 
                                                   udf['ASIN'],
                                                   udf['rating']))

In [ ]:
Train, Test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)
model = LightFM(loss='warp', no_components=50)
model.fit(Train, epochs=30, num_threads=1)

In [ ]:
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

k_values = range(1, 11)
auc_scores = []
precision_scores = []
recall_scores = []

for k in tqdm(k_values, desc="Processing Train"):
    auc = auc_score(model, Train).mean()
    precision = precision_at_k(model, Train, k=k).mean()
    recall = recall_at_k(model, Train, k=k).mean()

    auc_scores.append(auc)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("Train Results:")
for k, auc, precision, recall in zip(k_values, auc_scores, precision_scores, recall_scores):
    print(f'K = {k}: AUC score: {auc}, Precision: {precision}, Recall: {recall}')

In [ ]:
k_values = range(1, 11)
auc_scores = []
precision_scores = []
recall_scores = []

for k in tqdm(k_values, desc="Processing Test"):
    auc = auc_score(model, Test).mean()
    precision = precision_at_k(model, Test, k=k).mean()
    recall = recall_at_k(model, Test, k=k).mean()
    
    auc_scores.append(auc)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("Test Results:")
for k, auc, precision, recall in zip(k_values, auc_scores, precision_scores, recall_scores):
    print(f'K = {k}: AUC score: {auc}, Precision: {precision}, Recall: {recall}')

In [ ]:
def get_top_n_recommendations_with_latest_transactions(model, dataset, user_id, transactions_df, n=10):
    try:
        transactions_df.reset_index(inplace=True)
    except:
        pass

    # Generate predictions for all items for this user
    user_index = dataset.mapping()[0][user_id]
    
    # Retrieve the actual item indices from the dataset mapping
    item_indices = list(dataset.mapping()[2].values())  # This ensures you're using the correct item indices
    
    # Generate predictions using the correct item indices
    scores = model.predict(user_index, np.array(item_indices))
    
    # Map scores to item IDs
    reverse_item_map = {v: k for k, v in dataset.mapping()[2].items()}
    scored_items = [(score, reverse_item_map[i]) for score, i in zip(scores, item_indices)]
    
    # Proceed with filtering scores for latest items and ranking them as before
    latest_transactions = transactions_df[transactions_df['reviewerID'] == user_id]\
                            .sort_values('reviewDate', ascending=False)\
                            .head(10)
    latest_items = set(latest_transactions['ASIN'])
    latest_scores = [score for score in scored_items if score[1] in latest_items]
    latest_scores.sort(reverse=True, key=lambda x: x[0])  # Sort based on scores
    top_item_ids = [item for _, item in latest_scores[:n]]

    return top_item_ids

recommendations_lightfm = {}
for uid in userids:
    recommendations_lightfm[uid] = get_top_n_recommendations_with_latest_transactions(
        model, dataset, uid, reviews_df, n=10)

# Compare NDCG across algorithms

In [ ]:
df_surprise_ndcg = df_metrics_surprise.loc[[
    ("KNNBasic", "ndcg"), ("KNNWithMeans", "ndcg"), ("SVD", "ndcg")
    ]]
df_surprise_ndcg.rename(index={
    ("KNNBasic", "ndcg"): "KNNBasic",
    ("KNNWithMeans", "ndcg"): "KNNWithMeans",
    ("SVD", "ndcg"): "SVD"
    }, inplace=True)
df_surprise_ndcg = df_surprise_ndcg["average"]

ndcg_dict = {
    "KNNBasic": df_surprise_ndcg.loc["KNNBasic"],
    "KNNWithMeans": df_surprise_ndcg.loc["KNNWithMeans"],
    "SVD": df_surprise_ndcg.loc["SVD"],
    "ALS": ndcg_als,
    "Content-based": ave_ndcg,
}

ndcg_df = pd.DataFrame.from_dict(ndcg_dict, orient="index")
ndcg_df.rename(columns={0: "NDCG"}, inplace=True)
display(ndcg_df)

ndcg_df.plot(kind='bar', figsize=(10, 5))
plt.title("NDCG across different algorithms")
plt.show()

### Interpretation of Results

- The AUC score is consistently high (0.9986754059791565) across all K values, indicating excellent overall model performance in distinguishing between positive and negative interactions.

- Precision at K measures the proportion of recommended items in the top-K set that are relevant. Precision decreases as K increases in your results, which is expected. When you recommend more items (increase K), the likelihood of including irrelevant items increases, thus lowering precision. However, a precision above 0.08 even at K=10 is quite good, suggesting that the top recommended items are largely relevant.

- Recall at K measures the proportion of relevant items that are successfully recommended in the top-K set. Your results show increasing recall with K, which is typical. As you recommend more items, you're more likely to cover a higher fraction of the relevant items, hence the recall increases. A recall of 0.8549128186215507 at K=10 indicates that about 85% of the relevant items are captured in the top 10 recommendations.

## Sample Recommendations Per Algorithm

In [ ]:
non_nan_count = df_utility.notna().sum(axis=1)
df_top_reviewers = df_utility.loc[non_nan_count.sort_values(ascending=False).index].iloc[:20]
top_uids = df_top_reviewers.index.tolist()

In [86]:
try:
    reviews_df.set_index("reviewerID", inplace=True)
except:
    pass
df_list_of_recos = []
uid_recos = {}
pd.set_option('display.max_colwidth', 800)
for uid in top_uids:
    print(f"User history of {uid}")
    user_hist_df = reviews_df.loc[uid][['ASIN', 'ProductName']]
    display(user_hist_df)
    uid_recos[uid.split('_')[0]] = {
        'KNNBasic': [],
        'KNNWithMeans': [],
        'SVD': [],
        'ALS': [],
        'Content-based': []
    }
    uid_recos[uid.split('_')[0]]["KNNBasic"] = (user_hist_df, recommendations_surprise['KNNBasic'][uid])
    uid_recos[uid.split('_')[0]]["KNNWithMeans"] = (user_hist_df, recommendations_surprise['KNNWithMeans'][uid])
    uid_recos[uid.split('_')[0]]["SVD"] = (user_hist_df, recommendations_surprise['SVD'][uid])
    uid_recos[uid.split('_')[0]]["ALS"] = (user_hist_df, list(recommendations_als[uid].keys()))
    uid_recos[uid.split('_')[0]]["Content-based"] = (user_hist_df,list(recommendations_content['euc'][uid].keys()))
    uid_recos[uid.split('_')[0]]["LightFM"] = (user_hist_df, recommendations_lightfm[uid])

    df_dict = {}
    for algo in uid_recos[uid.split('_')[0]]:
        df_dict[algo] = asins_df.loc[list(uid_recos[uid.split('_')[0]][algo][1])]['name'].tolist()
    df_recos_across_algos = pd.DataFrame.from_dict(df_dict, orient='index').T
    df_list_of_recos.append(df_recos_across_algos)
    print(f"Recommendations for {uid}")
    display(df_recos_across_algos)

User history of Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA


,ASIN,ProductName
reviewerID,,
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b0b9bkq8s9,oghom_steamer_for_clothes_steamer_handheld_clothing_steamer_for_garment_240ml_portable_travel_steam_iron
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b0037zaxxy,gillette_fusion_proglide_sensitive_2_in_1_shave_gel_ocean_breeze_pack_of_2_12_oz_total
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b01mssdepk,cerave_hydrating_facial_cleanser_|_moisturizing_nonfoaming_face_wash_with_hyaluronic_acid_ceramides_and_glycerin_|_fragrance_free_paraben_free_|_16_fluid_ounce
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b09mm7bh9p,giantex_gx24977pe_mini_washer_and_spinner_combo_55lbs_capacity_semiautomatic_compact_laundry_apartment_dorm_rv_portable_washing_machine
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b0ct97ptsf,champion_men's_tshirt_cotton_midweight_men's_crewneck_tee_tshirt_for_men_script_reg_or_big_tall
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b07dtnx5w5,scotties_everyday_comfort_facial_tissues_92_tissues_per_box_4_pack_92_count_pack_of_4
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b07nn4nrrj,kleenex_ultra_soft_facial_tissues_65_count_pack_of_4_260_total_tissues
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b010ovk234,febreze_air_effects_air_freshener_linen_sky_88_oz_pack_of_6
Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA,b014injct4,lamicall_cell_phone_stand_desk_phone_holder_cradle_compatible_with_phone_12_mini_11_pro_xs_max_xr_x_8_7_6_plus_se_all_smartphones_charging_dock_office_desktop_accessories__silver


Recommendations for Real Deal_AE7Z57LXPIPMXRIFXY6673V6HTDA


,KNNBasic,KNNWithMeans,SVD,ALS,Content-based,LightFM
0,"Kolbs Gel Extreme | Wheelchair Cushion Seat Cushion | 22 x 18 Inch Seat, 3 Inch Thick | Coccyx, Sciatica & Tailbone Pain Relief Cushion | Seat Pad Ideal for Office, Kitchen Wheelchairs and Auto",ExpertPower 12v 7ah Rechargeable Sealed Lead Acid Battery,"Champion Men's T-shirt, Cotton Midweight Men's Crewneck Tee, T-shirt for Men, Script (Reg. or Big & Tall)",WERFORU No Buckle Stretch Belt for Women and Men Elastic Waist Belt up to 72 Inches for Jeans Pants,"VELSCRUN 6 Pack Spa Headband, Makeup Headband, Face Wash Headband, Skincare Headbands, Brown Sponge Hair Band Bow Tie Spa Headband for Washing Face Wristband Set Hair Accessories for Women Gifts","Febreze AIR Effects Air Freshener Linen & Sky, 8.8 oz (Pack of 6)"
1,"Gillette Fusion ProGlide Sensitive 2 in 1 Shave Gel, Ocean Breeze, Pack of 2, 12 oz Total","Giantex GX24977-PE, Mini Washer and Spinner Combo, 5.5lbs Capacity, Semi-automatic Compact Laundry Apartment Dorm RV Portable Washing Machine","Scotties Everyday Comfort Facial Tissues, 92 Tissues per Box, 4 Pack, 92 Count (Pack of 4)","LEGO Creator 3 in 1 Exotic Parrot Building Toy Set, Transforms to 3 Different Animal Figures - from Colorful Parrot, to Swimming Fish, to Cute Frog, Creative Toys for Kids Ages 7 and Up, 31136","BULLIANT Men's Belt,Brand Ratchet Belt Of Genuine Leather For Gift Men Dress,Size Customized","Champion Men's T-shirt, Cotton Midweight Men's Crewneck Tee, T-shirt for Men, Script (Reg. or Big & Tall)"
2,"BUZIO Insulated Water Bottle 64 oz with Straw Lid (3 Lids), 64oz Stainless Steel Bottle Half Gallon Jug Flask, Double Wall Vacuum Sports Thermo Mug, Cold Hot Hydro Metal Canteen, Indigo Black",RGB Headphone Stand with Wireless Charger Desk Gaming Headset Holder Hanger Rack with 10W/7.5W Fast Charge QI Wireless Charging Pad - Suitable for Gamer Desktop Table Game Earphone Accessories,"OGHom Steamer for Clothes Steamer, Handheld Clothing Steamer for Garment, 240ml Portable Travel Steam Iron","Dual USB 3.0 and USB-C Memory Card Reader - Supports SD, Micro SD, SDXC, SDHC, and More. Compatible with MacBook, iPad, Galaxy S21, and More.",12 Pieces Spring Flowers Car Tissues Box Spring Floral Car Tissues Cylinder in 4 Design Refill Round Car Tissue Holder with Facial Tissues for Wedding Birthday Party Car Cup Holder Travel Home,"OGHom Steamer for Clothes Steamer, Handheld Clothing Steamer for Garment, 240ml Portable Travel Steam Iron"
3,RGB Headphone Stand with Wireless Charger Desk Gaming Headset Holder Hanger Rack with 10W/7.5W Fast Charge QI Wireless Charging Pad - Suitable for Gamer Desktop Table Game Earphone Accessories,"Kleenex Ultra Soft Facial Tissues, 65 Count (Pack of 4) (260 Total Tissues)","CeraVe Hydrating Facial Cleanser | Moisturizing Non-Foaming Face Wash with Hyaluronic Acid, Ceramides and Glycerin | Fragrance Free Paraben Free | 16 Fluid Ounce","CloSYS Sensitive Mouthwash, 32 Ounce, Gentle Mint, Alcohol Free, Dye Free, pH Balanced, Helps Soothe Mouth Sensitivity, Fights Bad Breath","Car Tissue Refill, 12 Packs 50 Sheets per Pack Disposable Facial Tissue, Napkin, Travel Size Pack for Car Visor Holder Tissue Cylinder Holder or Purse, 12 Packs, 600 Sheets In Total","Kleenex Ultra Soft Facial Tissues, 65 Count (Pack of 4) (260 Total Tissues)"
4,"CeraVe Hydrating Facial Cleanser | Moisturizing Non-Foaming Face Wash with Hyaluronic Acid, Ceramides and Glycerin | Fragrance Free Paraben Free | 16 Fluid Ounce","Scotties Everyday Comfort Facial Tissues, 92 Tissues per Box, 4 Pack, 92 Count (Pack of 4)","Gildan Fleece Crewneck Sweatshirt, Style G18000, Multipack","Gain + Aroma Boost Laundry Detergent Liquid Soap, Island Fresh Scent, 45 Loads, 65 Fl Oz, He Compatible","Kleenex Perfect Fit Facial Tissues, 3 count (Packaging May Vary)","Degree Men Clinical Protection Antiperspirant Deodorant 72-Hour Sweat & Odor Protection Clean Prescription-Strength Antiperspirant For Men with MotionSense Technology 1.7 oz, Pack of 2"
5,Amazon Essentials Men's 

User history of Marcy G._AE76BAELYPHA2Q6CIHRWEHZVVXMQ


...

,KNNBasic,KNNWithMeans,SVD,ALS,Content-based,LightFM
0,Shorty Thompson: U.S. Marshal: Sent To England: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 126),Jubal Stone: U.S. Marshal: Downwind Of Trouble: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 46),Shorty Thompson: U.S. Marshal: Sent To England: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 126),Grit and Gunpowder: A Historical Western Adventure Novel,Tress of the Emerald Sea: A Cosmere Novel (Secret Projects Book 1),Thunder on the Chisholm Trail: A Historical Western Adventure Novel
1,Jubal Stone: U.S. Marshal: Downwind Of Trouble: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 46),Shorty Thompson: U.S. Marshal: Sent To England: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 126),A Brave Outlaw's Burden: A Historical Western Adventure Novel,A Hunter's Unseen Shadow: A Historical Western Adventure Novel,Lapvona: A Novel,The Wronged Man's Reckoning: A Historical Western Adventure Novel
2,Wrath of the Lone Rider: A Historical Western Adventure Novel,Gunsmoke in Silver City: A Historical Western Adventure Novel,Outlaw on the Ranch: A Historical Western Adventure Novel,Rescuing the Kidnapped Heiress: A Historical Western Adventure Novel,"Outlander: A Novel (Outlander, Book 1)",Gunsmoke in Silver City: A Historical Western Adventure Novel
3,A Brave Outlaw's Burden: A Historical Western Adventure Novel,Jubal Stone: U.S. Marshal: Showdown At High Plains Oasis: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 44),Shorty Thompson: U.S. Marshal: Too Young To Kill: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 124),Brothers in Guns and Dust: A Historical Western Adventure Novel,SECOND SKIN if you love Fourth Wing then discover this enthralling fantasy novel,Shorty Thompson: U.S. Marshal: Too Young To Kill: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 124)
4,A Detective's Bloody Pursuit: A Historical Western Adventure Novel,Wrath of the Lone Rider: A Historical Western Adventure Novel,Jubal Stone: U.S. Marshal: Downwind Of Trouble: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 46),A Miner's Deadly Secret: A Historical Western Adventure Novel,The Book of Doors: A Novel,Outlaw on the Ranch: A Historical Western Adventure Novel
5,Kill That Senator: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 125),Shorty Thompson: U.S. Marshal: Too Young To Kill: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 124),Ambush in the Dusty Mines: A Historical Western Adventure Novel,A Trapper's Battle Hymn: A Historical Western Adventure Novel,The Night Market: A Standalone Fae Fantasy Forbidden Romance (A Bound By Ravens Novel),A Brave Outlaw's Burden: A Historical Western Adventure Novel
6,Shorty Thompson: U.S. Marshal: Too Young To Kill: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 124),Kill That Senator: A Western Adventure Novel (Shorty Thompson: U.S. Marshal: Tales of the Old West Book 125),Wrath of the Lone Rider: A Historical Western Adventure Novel,Tempting the Highland Devil: A Medieval Historical Romance Novel (Highland Devils Book 3),A Curse of Scales and Flame: A Magic and Marvels Novel (Magic & Marvels Book 1),Jubal Stone: U.S. Marshal: Showdown At High Plains Oasis: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 44)
7,Jubal Stone: U.S. Marshal: Showdown At High Plains Oasis: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 44),A Brave Outlaw's Burden: A Historical Western Adventure Novel,Jubal Stone: U.S. Marshal: Showdown At High Plains Oasis: A Western Adventure Novel (A Jubal Stone: U.S. Marshal Western Book 44),Beyond Honor (Book 1: Galaxy At War): A Military Sci-

User history of Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA


,ASIN,ProductName
reviewerID,,
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b0773zy26f,sceptre_24inch_professional_thin_1080p_led_monitor_99%_srgb_2x_hdmi_vga_buildin_speakers_machine_black_e248w19203r_series
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b0chhsfmrl,sceptre_new_27inch_gaming_monitor_100hz_1ms_displayport_hdmi_x2_100%_srgb_amd_freesync_buildin_speakers_eye_care_frameless_machine_black_2024_e275wfw100t
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b00mevitcy,optimum_nutrition_optimen_daily_multivitamin_for_men_immune_support_supplement_with_amino_acids_80_day_supply_240_count_packaging_may_vary
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b0854wl4g3,quilted_northern_ultra_plushr_toilet_paper_18_mega_rolls_=_72_regular_rolls_3ply_bath_tissue
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b09nw8pnh8,angel_soft_toilet_paper_48_mega_rolls_=_192_regular_rolls_2ply_bath_tissue
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b086129ysx,bedsure_pillow_cases_standard_size_100%_rayon_derived_from_bamboo_cooling_pillowcase_silky_soft_breathable_pillow_covers_with_envelope_closure_for_kids_black_gift_for_men_or_women_20x26_inches
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b0747zc625,tide_pods_liquid_laundry_detergent_soap_pacs_he_compatible_42_count_powerful_3in1_clean_in_one_step_original_scent
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b0bjmv9bxj,tide_pods_laundry_detergent_soap_pods_spring_meadow_scent_112_count
Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA,b00mh4qm1s,amazon_basics_8pack_9_volt_alkaline_performance_allpurpose_batteries_5year_shelf_life_packaging_may_vary


Recommendations for Adam Nelson_AHIL2JHWADD6Z5U277LX5WLGG2VA


,KNNBasic,KNNWithMeans,SVD,ALS,Content-based,LightFM
0,"Quilted Northern Ultra Plush® Toilet Paper, 18 Mega Rolls = 72 Regular Rolls, 3-Ply Bath Tissue","Sceptre 24-inch Professional Thin 1080p LED Monitor 99% sRGB 2x HDMI VGA Build-in Speakers, Machine Black (E248W-19203R Series)","Amazon Basics 8-Pack 9 Volt Alkaline Performance All-Purpose Batteries, 5-Year Shelf Life, Packaging May Vary","Logitech MK270 Wireless Keyboard And Mouse Combo For Windows, 2.4 GHz Wireless, Compact Mouse, 8 Multimedia And Shortcut Keys, For PC, Laptop - Black",Dropps Fabric Softener Pods: Lavender Eucalyptus | 32 Count | HE Compatible + All Washers | Cold Wash + All Temperatures | Softness + Freshness | Low Waste Packaging,"Tide PODS Liquid Laundry Detergent Soap Pacs, HE Compatible, 42 Count, Powerful 3-in-1 Clean in one Step, Original Scent"
1,"Angel Soft Toilet Paper, 48 Mega Rolls = 192 Regular Rolls, 2-Ply Bath Tissue","SheaMoisture Curl & Shine Conditioner Coconut & Hibiscus, for Thick, Curly Hair to Moisturize & Soften, 13 oz","Optimum Nutrition Opti-Men Daily Multivitamin for Men, Immune Support Supplement with Amino Acids, 80 Day Supply, 240 Count, (Packaging May Vary)","CASEKOO for iPhone 14 Case/iPhone 13 Case Crystal Clear, [Not Yellowing] [Military Grade Drop Protection] Shockproof Protective Phone Case for Women Men 6.1 inch, Clear","Snuggle Exhilarations in Wash Laundry Scent Booster Pacs, Island Hibiscus and Rainflower, 56 Count","Amazon Basics 8-Pack 9 Volt Alkaline Performance All-Purpose Batteries, 5-Year Shelf Life, Packaging May Vary"
2,"Tide PODS Liquid Laundry Detergent Soap Pacs, HE Compatible, 42 Count, Powerful 3-in-1 Clean in one Step, Original Scent","Sceptre New 27-inch Gaming Monitor 100Hz 1ms DisplayPort HDMI x2 100% sRGB AMD FreeSync Build-in Speakers, Eye Care Frameless Machine Black 2024 (E275W-FW100T)","Aureday 88’’ Flexible Phone Tripod with 14.95’’ Gooseneck, Bluetooth Remote and Phone Holder, Overhead Cell Phone Stand＆Selfie Stick for Video Recording, Compatible with iPhone Android Phone, Camera","Highwings 8K 10K HDMI Cable 48Gbps 6.6FT/2M, Certified Ultra High Speed HDMI® Cable Braided Cord-4K@120Hz 8K@60Hz, DTS:X, HDCP 2.2 & 2.3, HDR 10 Compatible with Roku TV/PS5/HDTV/Blu-ray",Method Fabric Softener; Beach Sage; 53.5 Ounces; 45 Loads; 1 pack; Packaging May Vary,"Bedsure Pillow Cases Standard Size, 100% Rayon Derived from Bamboo Cooling Pillowcase, Silky Soft & Breathable Pillow Covers with Envelope Closure for Kids, Black, Gift for Men or Women, 20x26 Inches"
3,"Tide PODS Laundry Detergent Soap Pods, Spring Meadow Scent, 112 count","Amazon Basics 8-Pack 9 Volt Alkaline Performance All-Purpose Batteries, 5-Year Shelf Life, Packaging May Vary","Angel Soft Toilet Paper, 48 Mega Rolls = 192 Regular Rolls, 2-Ply Bath Tissue","Amazon Basics 3 Speed Small Room Air Circulator Fan, 7-Inch Blade, Black, 6.3""D x 11.1""W x 10.9""H","Laundry Sauce - Luxury Scented Laundry Detergent Pods & Scent Booster Bundle (Egyptian Rose), Premium Laundry Washing Pods with High-End, Sophisticated Fragrances - Dye Free & HE Compatible","Tide PODS Laundry Detergent Soap Pods, Spring Meadow Scent, 112 count"
4,"SheaMoisture Curl & Shine Conditioner Coconut & Hibiscus, for Thick, Curly Hair to Moisturize & Soften, 13 oz","Quilted Northern Ultra Plush® Toilet Paper, 18 Mega Rolls = 72 Regular Rolls, 3-Ply Bath Tissue","Sceptre New 27-inch Gaming Monitor 100Hz 1ms DisplayPort HDMI x2 100% sRGB AMD FreeSync Build-in Speakers, Eye Care Frameless Machine Black 2024 (E275W-FW100T)","HaFundy LED Desk Lamp for Home,Office,Reading,Adjustable Eye-Caring Desk Light with Clamp,Swing Arm Lamp Includes 3 Color Modes,10 Brightness Levels Table Lamps with Memory Function(Black) (Black)",Dropps Active Wash Laundry Detergent Pods: Rose Energy | 24 Count | HE Compatible + All Washers | Cold Wash + All Temperatures | Sport Detergent | Low Waste Packaging,"Amazon Basics USB-A to USB-B 2.0 Cable for Printer or External Hard Drive, Gold-Plated Connectors, 1

## Build Pickle File For Streamlit Application

#### Export User History and Recommendations as Pickle file

In [ ]:
# Save the dictionary to a pickle file
pd.to_pickle(uid_recos, 'user_hist_reco.pkl')

# Results and Recommendations

## Results

The recommendations served by the algorithms are more personalized compared to the baseline global average. There is a clear connection between the user's history and characteristics with the algorithm's recommendations, unlike the recommendations served by the baseline global average, which essentially is just the top K most popular items.

In terms of metrics, the offline evaluation metrics for the different algorithms demonstrated above are all by definition "good". However, the researchers would like to stress that the real determining factor for whether a recommender system performs well or not is based on eventual online evaluation metrics. That said, the researchers still expect the various recommender systems to perform well. For instance, the vision is that the recommendations are two-pronged:

* *Customers who bought this item also bought*: User-based collaborative filtering, ALS
* *Recommended for you*: Content-based filtering / Hybrid recommender using LightFM

In any case, future work on this project can simply mix and match or swap out algorithms.

The exact metrics for each algorithm is as follows:


* KNNBasic	0.266200
* KNNWithMeans	0.263800
* SVD	0.266397
* ALS	0.997882
* Content-based with Euclidean distance	0.965205
* Content-based with Cosine distance	0.984095
* Content-based with Jaccard distance	0.951475

Here we see that most algorithms except for KNNBasic, KNNWithMeans, and SVD perform well on a ranking basis.

## Recommendations

To make the recommender system more robust, the researchers suggest the following:

* Include items that are less popular than the ones included in the current itemset
* Include reviewers with less reviews than the ones included in the current userbase
* Fully understand and implement a hybrid recommender with tuned hyperparameters. In the case of this project, LightFM and ALS were used but hyperparameter tuning was not implemented. Hence, there is a possibility that these algorithms can actually improve after tuning.